In [2]:
import os
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import subprocess
import re

def get_output(string):
    # Extract the classes from the string
    pattern = r"\d+\s+([a-zA-Z]+)"
    matches = re.findall(pattern, string)
    classes = matches[:9]  # Get the first 9 matches
    print(classes)

def run_yolov7(file_path):
    command = f'python detect.py --weights best.pt --conf 0.25 --img-size 640 --source "{file_path}" --no-trace'
    try:
        result = subprocess.check_output(command, shell=True).decode('utf-8')
        get_output(result)  # Print the output

        # Get the path to the folder with the maximum number in './runs/detect'
        folder_path = get_max_folder_path('./runs/detect')

        # Get the image name from the folder
        image_name = get_image_name(folder_path)

        # Get the full image path
        image_path = os.path.join(folder_path, image_name)

        # Display the image
        display_image(image_path)
    except subprocess.CalledProcessError as e:
        error_message = f"Error: Command '{command}' returned non-zero exit status {e.returncode}"
        print(error_message)

def get_max_folder_path(directory):
    folders = [folder for folder in os.listdir(directory) if os.path.isdir(os.path.join(directory, folder))]
    max_folder = max(folders, key=lambda x: int(x[3:]) if x[3:].isdigit() else -1)
    return os.path.join(directory, max_folder)

def get_image_name(folder_path):
    files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    image_files = [file for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if image_files:
        return image_files[0]
    else:
        return ''

def display_image(image_path):
    # Open the image using PIL
    img = Image.open(image_path)

    # Resize the image if needed
    img = img.resize((640, 600))

    # Create a Tkinter-compatible image object
    img_tk = ImageTk.PhotoImage(img)

    # Create a label in the Tkinter window to display the image
    image_label.config(image=img_tk)
    image_label.image = img_tk

def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png")])
    if file_path:
        run_yolov7(file_path)

# Create the tkinter window
window = tk.Tk()
window.geometry("640x600")  # Set the window size to 640x600

# Create a button to select the image
select_button = tk.Button(window, text="Select Image", command=select_image)
select_button.pack()

# Create a label to display the image
image_label = tk.Label(window)
image_label.pack()

# Start the tkinter event loop
window.mainloop()


['antinal', 'muco', 'panadol', 'other', 'telfast', 'maalox', 'concor', 'colona', 'cidophage']


In [8]:
import os
import tkinter as tk
from tkinter import filedialog
from tkinter.ttk import Progressbar
from PIL import ImageTk, Image
import subprocess
import re
import threading

def get_output(string):
    # Extract the classes from the string
    pattern = r"\d+\s+([a-zA-Z]+)"
    matches = re.findall(pattern, string)
    classes = matches[:9]  # Get the first 9 matches
    print(classes)

def run_yolov5(file_path):
    command = f'python detect.py --weights best.pt --conf 0.25 --img-size 640 --source "{file_path}" --no-trace'
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        
        # Update the progress bar while the process is running
        while process.poll() is None:
            progress_bar.step(10)
            window.update_idletasks()

        output = process.communicate()[0].decode('utf-8')
        get_output(output)  # Print the output

        # Get the path to the folder with the maximum number in './runs/detect'
        folder_path = get_max_folder_path('./runs/detect')

        # Get the image name from the folder
        image_name = get_image_name(folder_path)

        # Get the full image path
        image_path = os.path.join(folder_path, image_name)

        # Display the image
        display_image(image_path)

        # Hide the progress bar
        progress_bar.stop()
        progress_bar.grid_remove()
    except subprocess.CalledProcessError as e:
        error_message = f"Error: Command '{command}' returned non-zero exit status {e.returncode}"
        print(error_message)

def get_max_folder_path(directory):
    folders = [folder for folder in os.listdir(directory) if os.path.isdir(os.path.join(directory, folder))]
    max_folder = max(folders, key=lambda x: int(x[3:]) if x[3:].isdigit() else -1)
    return os.path.join(directory, max_folder)

def get_image_name(folder_path):
    files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    image_files = [file for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if image_files:
        return image_files[0]
    else:
        return ''

def display_image(image_path):
    # Open the image using PIL
    img = Image.open(image_path)

    # Resize the image if needed
    img = img.resize((640, 600))

    # Create a Tkinter-compatible image object
    img_tk = ImageTk.PhotoImage(img)

    # Create a label in the Tkinter window to display the image
    image_label.config(image=img_tk)
    image_label.image = img_tk

def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png")])
    if file_path:
        # Show the progress bar
        progress_bar.grid(row=2, column=0, columnspan=2)
        progress_bar.start(10)

        # Run YOLOv5 on the selected image in a separate thread
        thread = threading.Thread(target=run_yolov5, args=(file_path,))
        thread.start()

# Create the tkinter window
window = tk.Tk()
window.geometry("640x650")  # Set the window size to 640x650

# Create a button to select the image
select_button = tk.Button(window, text="Select Image", command=select_image)
select_button.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

# Create a progress bar
progress_bar = Progressbar(window, mode='indeterminate', length=300)
progress_bar.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

# Create a label to display the image
image_label = tk.Label(window)
image_label.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

# Start the tkinter event loop
window.mainloop()


['torch', 'CUDA', 'layers', 'parameters', 'gradients', 'GFLOPS', 'antinal', 'muco', 'panadol']


In [ ]:
import os
import tkinter as tk
from tkinter import filedialog
from tkinter.ttk import Progressbar
from PIL import ImageTk, Image
import subprocess
import re
import threading

def get_output(string):
    # Extract the classes from the string
    pattern = r"\d+\s+([a-zA-Z]+)"
    matches = re.findall(pattern, string)
    classes = matches[:9]  # Get the first 9 matches
    print(classes)

def run_yolov5(file_path):
    command = f'python detect.py --weights best.pt --conf 0.25 --img-size 640 --source "{file_path}" --no-trace'
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        
        total_lines = sum(1 for _ in process.stdout)  # Count the total number of lines in the output
        
        # Reset the progress bar
        progress_bar['value'] = 0
        progress_bar['maximum'] = total_lines

        for line in process.stdout:
            progress_bar.step()
            window.update_idletasks()

        output = process.communicate()[0].decode('utf-8')
        get_output(output)  # Print the output

        # Get the path to the folder with the maximum number in './runs/detect'
        folder_path = get_max_folder_path('./runs/detect')

        # Get the image name from the folder
        image_name = get_image_name(folder_path)

        # Get the full image path
        image_path = os.path.join(folder_path, image_name)

        # Display the image
        display_image(image_path)

        # Hide the progress bar
        progress_bar.stop()
        progress_bar.grid_remove()
    except subprocess.CalledProcessError as e:
        error_message = f"Error: Command '{command}' returned non-zero exit status {e.returncode}"
        print(error_message)

def get_max_folder_path(directory):
    folders = [folder for folder in os.listdir(directory) if os.path.isdir(os.path.join(directory, folder))]
    max_folder = max(folders, key=lambda x: int(x[3:]) if x[3:].isdigit() else -1)
    return os.path.join(directory, max_folder)

def get_image_name(folder_path):
    files = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    image_files = [file for file in files if file.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if image_files:
        return image_files[0]
    else:
        return ''

def display_image(image_path):
    # Open the image using PIL
    img = Image.open(image_path)

    # Resize the image if needed
    img = img.resize((640, 600))

    # Create a Tkinter-compatible image object
    img_tk = ImageTk.PhotoImage(img)

    # Create a label in the Tkinter window to display the image
    image_label.config(image=img_tk)
    image_label.image = img_tk

def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png")])
    if file_path:
        # Show the progress bar
        progress_bar.grid(row=2, column=0, columnspan=2)
        progress_bar.start(10)

        # Run YOLOv5 on the selected image in a separate thread
        thread = threading.Thread(target=run_yolov5, args=(file_path,))
        thread.start()

# Create the tkinter window
window = tk.Tk()
window.geometry("640x650")  # Set the window size to 640x650

# Create a button to select the image
select_button = tk.Button(window, text="Select Image", command=select_image)
select_button.grid(row=0, column=0, columnspan=2, padx=10, pady=10)

# Create a progress bar
progress_bar = Progressbar(window, mode='determinate', length=300)
progress_bar.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

# Create a label to display the image
image_label = tk.Label(window)
image_label.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

# Start the tkinter event loop
window.mainloop()
